In [ ]:
import json
import pickle

In [2]:
instance_id = 'astropy__astropy-12907'
REPO_GRAPH_LOC = (
    '/home/gangda/workspace/auto-search-agent/graph_encoder/DATA/repo_graphs'
)
PROJECT_FILE_LOC = (
    '/home/gangda/workspace/auto-search-agent/graph_encoder/DATA/repo_structures'
)

G = pickle.load(open(f'{REPO_GRAPH_LOC}/graph/{instance_id}.pkl', 'rb'))
graph_tags = json.load(open(f'{REPO_GRAPH_LOC}/tags/tags_{instance_id}.json', 'r'))
project_structure = json.load(open(f'{PROJECT_FILE_LOC}/{instance_id}.json', 'r'))

In [7]:
def filter_none_python(structure):
    for key, value in list(structure.items()):
        if (
            'functions' not in value.keys()
            and 'classes' not in value.keys()
            and 'text' not in value.keys()
        ) or not len(value.keys()) == 3:
            filter_none_python(value)

            if structure[key] == {}:
                del structure[key]
        else:
            if not key.endswith('.py'):
                del structure[key]


def filter_out_test_files(structure):
    """filter out test files from the project structure"""
    for key, value in list(structure.items()):
        if key.startswith('test'):
            del structure[key]
        elif isinstance(value, dict):
            filter_out_test_files(value)


struct = project_structure['structure']
filter_none_python(struct)
if not project_structure['instance_id'].startswith('pytest'):
    filter_out_test_files(struct)

In [7]:
search_term = 'ConfigItem'
list(G.neighbors(search_term))

['__init__',
 '__set__',
 '__get__',
 'set',
 'set_temp',
 'reload',
 '__repr__',
 '__str__',
 '__call__',
 '_validate_val',
 '',
 'ConfigItem']

In [8]:
found_loc = [
    (
        'astropy/modeling/separable.py',
        'function: separability_matrix\nfunction: _coord_matrix\nfunction: _cstack\nfunction: _cdot\nfunction: _separable',
    ),
    (
        'astropy/modeling/core.py',
        'class: CompoundModel\nfunction: CompoundModel._make_leaflist\nfunction: CompoundModel.traverse_postorder\nfunction: CompoundModel._format_expression\nfunction: CompoundModel._format_components\nfunction: CompoundModel._set_sub_models_and_parameter_units',
    ),
]

for path, item in found_loc:
    print(item.splitlines())

['function: separability_matrix', 'function: _coord_matrix', 'function: _cstack', 'function: _cdot', 'function: _separable']
['class: CompoundModel', 'function: CompoundModel._make_leaflist', 'function: CompoundModel.traverse_postorder', 'function: CompoundModel._format_expression', 'function: CompoundModel._format_components', 'function: CompoundModel._set_sub_models_and_parameter_units']


In [35]:
from collections import Counter

tags, names = [], []
for tag in graph_tags:
    if tag['kind'] == 'def':
        tags.append(tag)
        names.append(tag['name'])

for tag in tags:
    if tag['name'] == 'token':
        print(tag['info'])

Counter(names)

Counter({'evaluate': 79,
         'assert_equal': 59,
         'from_tree_transform': 51,
         'to_tree_transform': 51,
         'input_units': 49,
         'inverse': 47,
         '_parameter_units_for_data_units': 42,
         '__call__': 40,
         'fit_deriv': 37,
         'parse': 34,
         'write': 27,
         'data': 25,
         'name': 24,
         'unit': 24,
         'ref': 21,
         'bounding_box': 21,
         'validate': 20,
         '__enter__': 18,
         '__exit__': 18,
         'value': 18,
         'from_tree': 18,
         'to_string': 16,
         'to_tree': 16,
         'transform': 15,
         'match_header': 15,
         'Conf': 14,
         'wrapper': 14,
         '__set__': 13,
         '__array_finalize__': 13,
         'represent_as': 13,
         'to_cartesian': 13,
         'from_cartesian': 13,
         'set_jds': 13,
         '_scale_operation': 12,
         'to_value': 12,
         '__getattr__': 12,
         'get_extensions': 11,
      

In [36]:
tags, names = [], []
for tag in graph_tags:
    if tag['kind'] == 'ref' and tag['name'] == 'token':
        tags.append(tag)
        names.append(tag['name'])

In [39]:
tags

[{'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/cpp.py',
  'rel_fname': 'astropy/extern/ply/cpp.py',
  'line': 185,
  'name': 'token',
  'kind': 'ref',
  'category': 'function',
  'info': '            tok = self.lexer.token()\n'},
 {'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/cpp.py',
  'rel_fname': 'astropy/extern/ply/cpp.py',
  'line': 212,
  'name': 'token',
  'kind': 'ref',
  'category': 'function',
  'info': '        tok = self.lexer.token()\n'},
 {'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/cpp.py',
  'rel_fname': 'astropy/extern/ply/cpp.py',
  'line': 220,
  'name': 'token',
  'kind': 'ref',
  'category': 'function',
  'info': '        tok = self.lexer.token()\n'},
 {'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/cpp.py',
  'rel_fname': 'astropy/extern/ply/cpp.py',
  'line': 229,
  'name': 'token',
  'kind': 'ref',
  'cate

    def token(self):
        try:
            while True:
                tok = next(self.parser)
                if tok.type not in self.ignore: return tok
        except StopIteration:
            self.parser = None
            return None


    def token(self):
        # Make local copies of frequently referenced attributes
        lexpos    = self.lexpos
        lexlen    = self.lexlen
        lexignore = self.lexignore
        lexdata   = self.lexdata

        while lexpos < lexlen:
            # This code provides some short-circuit code for whitespace, tabs, and other ignored characters
            if lexdata[lexpos] in lexignore:
                lexpos += 1
                continue

            # Look for a regular expression match
            for lexre, lexindexfunc in self.lexre:
                m = lexre.match(lexdata, lexpos)
                if not m:
                    continue

                # Create a token for return
                tok = LexToken()
                t

Def -> Node feature
Ref -> Edge

In [32]:
G.nodes['token']

{'category': 'function',
 'info': 'def token():\n    warnings.warn(_warnmsg)\n    return _token()\n\n',
 'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/yacc.py',
 'line': 181,
 'kind': 'def'}

In [40]:
# for tag in tags:
tag = tags[0]
path = tag['rel_fname'].split('/')
print(path)
s = struct
for p in path:
    s = s[p]

['astropy', 'extern', 'ply', 'cpp.py']


In [1]:
s['text']

NameError: name 's' is not defined

In [153]:
s['classes'][0]

{'name': 'Conf',
 'start_line': 44,
 'end_line': 71,
 'text': ['class Conf(_config.ConfigNamespace):',
  '    """',
  '    Configuration parameters for `astropy`.',
  '    """',
  '',
  '    unicode_output = _config.ConfigItem(',
  '        False,',
  "        'When True, use Unicode characters when outputting values, and '",
  "        'displaying widgets at the console.')",
  '    use_color = _config.ConfigItem(',
  "        sys.platform != 'win32',",
  "        'When True, use ANSI color escape sequences when writing to the console.',",
  "        aliases=['astropy.utils.console.USE_COLOR', 'astropy.logger.USE_COLOR'])",
  '    max_lines = _config.ConfigItem(',
  '        None,',
  "        description='Maximum number of lines in the display of pretty-printed '",
  "        'objects. If not provided, try to determine automatically from the '",
  "        'terminal size.  Negative numbers mean no limit.',",
  "        cfgtype='integer(default=None)',",
  "        aliases=['astropy.ta

In [127]:
struct.keys()

dict_keys(['astropy', 'docs', 'examples', '.pyinstaller'])

In [45]:
G.nodes['token']

{'category': 'function',
 'info': 'def token():\n    warnings.warn(_warnmsg)\n    return _token()\n\n',
 'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/yacc.py',
 'line': 181,
 'kind': 'def'}

In [62]:
list(G.predecessors('autopower'))

['BoxLeastSquares', 'LombScargle', 'autopower']

In [63]:
list(G.successors('autopower'))

['autopower']

In [9]:
tags = []
for tag in graph_tags:
    if tag['name'] == 'token' and tag['kind'] == 'ref':
        tags.append(tag)

In [10]:
tag = tags[0]
print(tag)
path = tag['rel_fname'].split('/')
print(path)
s = struct
for p in path:
    s = s[p]

one_hop_tags = []
for txt in s['functions']:
    if txt['start_line'] <= tag['line'] <= txt['end_line']:
        one_hop_tags.append((txt, tag['rel_fname']))
for txt in s['classes']:
    for func in txt['methods']:
        if func['start_line'] <= tag['line'] <= func['end_line']:
            func['text'].insert(0, txt['text'][0])
            one_hop_tags.append((func, tag['rel_fname']))

{'fname': 'playground/b4bffafa-6319-41e4-a4f0-81e61108195c/astropy/astropy/extern/ply/cpp.py', 'rel_fname': 'astropy/extern/ply/cpp.py', 'line': 185, 'name': 'token', 'kind': 'ref', 'category': 'function', 'info': '            tok = self.lexer.token()\n'}
['astropy', 'extern', 'ply', 'cpp.py']


In [14]:
one_hop_tags[0]

({'name': 'tokenize',
  'start_line': 182,
  'end_line': 189,
  'text': ['class Preprocessor(object):',
   '    def tokenize(self,text):',
   '        tokens = []',
   '        self.lexer.input(text)',
   '        while True:',
   '            tok = self.lexer.token()',
   '            if not tok: break',
   '            tokens.append(tok)',
   '        return tokens']},
 'astropy/extern/ply/cpp.py')